In [6]:
# Import the classes.
from analysis.timing.graph import *
from analysis.timing.entities import *

# Import the timing analysis functions.
import analysis.timing.timing_analysis as analysis

# For weight functions.
import functools

# For logging.
import logging

# 
import csv

In [7]:
# Set the logging level.
analysis.set_logger_level(logging.ERROR)


# Network Support

In [8]:
class Device(Node):
    def __init__(self, id, entities, base_delay=None):
        Node.__init__(self, id)
        self.entities = entities
        self.base_delay = base_delay

    def software_delay(self) -> float:
        if self.base_delay:
            return len(self.entities) * self.base_delay
        return 0


def find_containing_node(G: Graph, e: Entity) -> Device:
    for node in G.graph:
        if e in node.entities:
            return node
    return None


def get_delay(G: Graph, e0: Entity, e1: Entity):
    n0 = find_containing_node(G, e0)
    n1 = find_containing_node(G, e1)
    assert n0, "No node contains {}".format(e0)
    assert n1, "No node contains {}".format(e1)
    spath = G.find_shortest_path(n0, n1)
    assert spath, "Cannot find shortest path"
    print(spath)
    return n0.software_delay() + spath[0] + n1.software_delay()


def create_relation(G: Graph, e0: Entity, e1: Entity):
    return Relation(e0, e1, get_delay(G, e0, e1))


# Scenario (SOME/IP)

In [9]:
# Load the services.
S0 = Service("S0", boot_del=2, init_del=4, rep_del=1, rep_max=3, cyc_del=2, ans_del=1, offer_mode=True)
S1 = Service("S1", boot_del=1, init_del=3, rep_del=1, rep_max=3, cyc_del=2, ans_del=1, offer_mode=True)
S2 = Service("S2", boot_del=4, init_del=6, rep_del=1, rep_max=3, cyc_del=2, ans_del=1, offer_mode=True)
S3 = Service("S3", boot_del=2, init_del=4, rep_del=1, rep_max=3, cyc_del=2, ans_del=1, offer_mode=True)
S4 = Service("S4", boot_del=1, init_del=3, rep_del=1, rep_max=3, cyc_del=2, ans_del=1, offer_mode=True)

# Load the clients.
C0 = Client("C0", boot_del=3, init_del=2, rep_del=1, rep_max=3, find_mode=True)
C1 = Client("C1", boot_del=3, init_del=2, rep_del=1, rep_max=3, find_mode=True)
C2 = Client("C2", boot_del=3, init_del=2, rep_del=1, rep_max=3, find_mode=True)
C3 = Client("C3", boot_del=3, init_del=2, rep_del=1, rep_max=3, find_mode=True)
C4 = Client("C4", boot_del=3, init_del=2, rep_del=1, rep_max=3, find_mode=True)
C5 = Client("C5", boot_del=3, init_del=2, rep_del=1, rep_max=3, find_mode=True)


# Scenario (Graph)

In [10]:
# Place them inside nodes.
n0 = Device(0, [S0, C0, C1], 0.05)
n1 = Device(1, [S1, S2], 0.025)
n2 = Device(2, [S3, C2, C3], 0.075)
n3 = Device(3, [S4, C4], 0.045)
n4 = Device(4, [C5], 0.076)

# 
def weight_function(G:Graph, n0:Node, n1:Node, frame_length:float, link_speed:float) -> float:
    return (frame_length * 8) / link_speed

# Create the graph.
G = Graph(
    [
        (n0, n1, functools.partial(weight_function, frame_length=256, link_speed=1e09)),
        (n1, n2, functools.partial(weight_function, frame_length=256, link_speed=1e06)),
        (n1, n3, functools.partial(weight_function, frame_length=1024, link_speed=1e03)),
        (n3, n4, functools.partial(weight_function, frame_length=256, link_speed=1e09)),
        (n2, n3, functools.partial(weight_function, frame_length=256, link_speed=1e06)),
        (n2, n4, functools.partial(weight_function, frame_length=1024, link_speed=1e03))
    ],
    False
)


KeyError: 0

In [ ]:
# Define the relations between clients and services.
system = [
    create_relation(G, C0, S3),
    create_relation(G, C0, S4),
    create_relation(G, C1, S0),
    create_relation(G, C2, S4),
    create_relation(G, C3, S4),
    create_relation(G, C3, S0),
    create_relation(G, C4, S1),
    create_relation(G, C5, S0)
]

# Compute the worst-case discovery time.
analysis.timing_analysis_system(system)


14.420050048000002

# Graph to CSV

In [ ]:
def write_to_csv():
    pass

def read_from_csv():
    pass
